# 교차 검증 (cross validation)
- 이전 결정트리에서는 depth조절을 직접해주어 테스트셋에 대해 성능향상을 유도했다. 하지만 여기서 찾아낸 depth값은 현재 가지고있는 *테스트셋*에만 최적화된 값이다. 실제 세상의 데이터는 예측불가하기 때문에 에측성능이 그대로 유지가 안될 수도 있다. 
  - 사람이 직접 설정해주는 파라미터들을 *하이퍼 파라미터*라고한다.
- 테스트셋을 실제 세상의 데이터라고 생각하고, 학습이 완전히 끝난 후 1번만 사용해야, 학습모델이 테스트셋(실제데이터)에 대해 정확히 성능을 측정할수있다.

# 검증 세트
- 테스트
- 모델을 훈련시키고 성능을 검증할 떄, 테스트셋을 사용하지 않으려면 어떻게해야할까? 기존 훈련셋을 또 나누면 된다. 
  - 만약 처음에 훈련셋:테스트셋=0.8:0.2 로 나누었으면 그중 0.8을 다시 훈련셋:검증셋=0.75:0.25 로 나누는것이다.

In [4]:
import pandas as  pd
wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine[['class']].to_numpy()
print(data)
print(target)

[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]]
[[0.]
 [0.]
 [0.]
 ...
 [1.]
 [1.]
 [1.]]


In [8]:
from sklearn.model_selection import train_test_split

# 훈련:테스트 분리 
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# 훈련:검증 분리 
X_train2, X_valid, y_train2, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [11]:
print(X_train.shape)
print(X_train2.shape)
print(X_valid.shape)
print(X_train2.shape[0] + X_valid.shape[0] == X_train.shape[0])

(5197, 3)
(4157, 3)
(1040, 3)
True


In [14]:
from sklearn.tree import DecisionTreeClassifier

# 훈련셋으로 결정트리 모델 훈련
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train2, y_train2)

print(f'훈련셋 정확도 {dt.score(X_train2, y_train2)}')
print(f'검증셋 정확도 {dt.score(X_valid, y_valid)} \n overfitting 되었다.')

훈련셋 정확도 0.9971133028626413
검증셋 정확도 0.864423076923077


- 훈련셋을 1번 더 나누니 훈련셋의 크기가 줄어들었다. 보통 모델이 훈련하는 데이터가 다양할수록 성능이 좋아진다. 
- 지금 같이 적은 데이터로 다양한 데이터를 훈련시키려는 경우 교차검증을 이용하면 된다. 교차검증은 훈련셋 일부를 검증셋으로 여러번 떼어내어 평가하는 과정을 여러번 반복한다.
  - 데이터가 99개일 때, 3개로 분리하여 교차검증 -> K-Fold cross validation (여기에서 K는 3이다)
    1. 33(훈련):33(훈련):33(**검증**)
    2. 33(훈련):33(**검증**):33(훈련)
    3. 33(**검증**):33(훈련):33(검증)


In [26]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, X_train, y_train)
print(f'훈련하는 시간 배열 {scores["fit_time"]}')
print(f'검증하는 시간 배열 {scores["score_time"]}')
print(f'테스트셋 정확도2 배열 {scores["score_time"]}')
print(f'배열의 길이를 보니 K의 기본값은 5인걸 알 수 있다.')

훈련하는 시간 배열 [0.00835466 0.00720167 0.00749612 0.00747013 0.00732589]
검증하는 시간 배열 [0.00076294 0.00071597 0.0007441  0.00076365 0.00073409]
테스트셋 정확도2 배열 [0.00076294 0.00071597 0.0007441  0.00076365 0.00073409]


- cross_valide()는 나눠주기만하고 훈련세트를 섞어주지는 않는다. 섞기위해서는 분할기를 지정해주어야한다.
  - 회귀 모델
    - KFold 분할기 사용
  - 분류 모델
    - StratifiedKFold 사용

In [30]:
from sklearn.model_selection import StratifiedKFold
import numpy as np 

scores = cross_validate(dt, X_train, y_train, cv=StratifiedKFold()) # 와인의 종류를 맞춘다 -> 분류모델 -> StratifiedKFold 사용
print(f'5-Fold 평균성능 {np.mean(scores["test_score"])}')
scores = cross_validate(dt, X_train, y_train, cv=StratifiedKFold( n_splits=10, shuffle=True, random_state=42),) # 와인의 종류를 맞춘다 -> 분류모델 -> StratifiedKFold 사용
print(f'10-Fold 평균성능 {np.mean(scores["test_score"])}')

5-Fold 평균성능 0.855300214703487
10-Fold 평균성능 0.8574181117533719


# 하이퍼 파라미터  튜닝
- 사람이 설정해주는 하이퍼 파라미터값의 최적값을 찾으려면 수많은 경우의 수를 테스트해봐야한다. 개발자가 직접 for문을 사용해 구현해볼수도  있지만, 사이킷런에서는 **Grid Search**라는것을 제공한다
- Grid Search는 하이퍼 파라미터 탐색과 교차검증을 한번에 해준다.

In [36]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# DecisionTreeClassifier에  사용할 파라미터를 5길이의 배열로 전달
# GridSearchCV의 반복 기본값 5임
# 5 * 5 = 25개의 모델을 만들어 내어 훈련함 -> 그중 가장 좋은 모델을 찾아 사용할것이다.
# n_jobs는 이 훈련에 사용될 코어의 갯수르 지정해주는 파라미터 (-1은 모든 코어를 뜻함)
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

dt = gs.best_estimator_
print(dt.score(X_train, y_train))
print(gs.best_params_)

0.9615162593804117
{'min_impurity_decrease': 0.0001}


In [37]:
# 가장 평균점수가 좋았던 Fold의 점수
print(gs.cv_results_['mean_test_score'])

# 파라미터를 여러개 전달하는것도 가능하며, 파라미터들의 갯수를 모두 곱한값에 * 5 만큼의 갯수의 모델을 훈련하게 된다.
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
  
# 하지만 역시 여기서도 파라미터의 범위를 사람이 직접 입력하고 있다.

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


### 랜덤서치
- 사람이 직접 파라미터값을 만들어서 전달하는것이 아닌, 범위만 지정해주고 값은 그 안에서 랜덤한 값들을 추출하여 전달하는것

In [51]:
from scipy.stats import uniform, randint
rgen = randint(0, 10) # 0 <= x < 10 정수타입 난수값 생성기
ugen = uniform(0, 1) # 0 <= x < 1 실수타입 난수값 생성기

np.unique(rgen.rvs(1000), return_counts=True)  # 1000개 생성하고 각자 몇번 나왔는지 확인

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, ), params, 
                        # 총 100번만 학습
                        n_iter=100, n_jobs=-1, random_state=42, )
gs.fit(X_train, y_train)

print(gs.best_params_)
print(gs.best_estimator_.score(X_test, y_test))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.86


- 컴퓨팅 파워가 좋아야 이런 작업들을 빨리 수행할 수 있다. 만약 이미지같은 비정형데이터를 학습시키려고한다면 지금보다 훨씬 강한 컴퓨팅파워와 시간이 필요할것이다. 